In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
# OLS
from statsmodels.formula.api import ols
# VIF 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.metrics import mean_absolute_error

In [68]:
total = pd.read_csv("total_data(2016포함).csv")

In [69]:
total = total[(total["year"] != "2020(상반기)") & (total["year"] != "2020(하반기)")]

In [70]:
col_name_x = total.columns[2:-5]
col_name_y = total.columns[-5:]

In [71]:
X = total[col_name_x]
y = total[col_name_y]
x_train = X.loc[:163]
x_test = X.loc[164:]
y_train = y.loc[:163]
y_test = y.loc[164:]

In [72]:
y1_train = y_train[['q1']]
y1_test = y_test[['q1']]

y2_train = y_train["q2"]
y2_test = y_test["q2"]

y3_train = y_train["q3"]
y3_test = y_test["q3"]

y4_train = y_train["q4"]
y4_test = y_test["q4"]

y5_train = y_train["q5"]
y5_test = y_test["q5"]

In [73]:
std_scaler = StandardScaler()
std_scaler.fit(x_train)
x_train_s = pd.DataFrame(std_scaler.transform(x_train), columns = x_train.columns)
x_train_s = sm.add_constant(x_train, has_constant = "add")

x_test_s = std_scaler.transform(x_test)
x_test_s = pd.DataFrame(std_scaler.transform(x_test), columns = x_test.columns)
x_test_s = sm.add_constant(x_test, has_constant = "add")

In [93]:
b = x_train_s.columns

In [98]:
b = ['for_4050', 'for_합계', 'for_u20', 'for_2030', 'single_2030',
       'single_f_o60', 'single_o60', 'single_f_합계', 'popu_2030', '가해_10대이하',
       'theft_violence_y', 'popu_o60', 'single_합계', 'robber_murder_y',
       'for_o60', 'report_sx_여성', '피해_80대이상', 'single_f_4050', 'vio_cnt',
       'robber_murder_x']

In [99]:
## 전진 단계별 선택법
variables = x_train_s[b].columns.tolist()
y = y1_train
selected_variables = [] ## 선택된 변수들
sl_enter = 0.05
sl_remove = 0.05
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index=remainder) ## 변수의 p-value
    ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서
    ## 선형 모형을 적합한다.
    for col in remainder:
        X = x_train_s[selected_variables + [col]]
        X = sm.add_constant(X)
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
    min_pval = pval.min()
    if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
        selected_variables.append(pval.idxmin())
        ## 선택된 변수들에대해서
        ## 어떤 변수를 제거할지 고른다.
        while len(selected_variables) > 0:
            selected_X = x_train_s[selected_variables]
            selected_X = sm.add_constant(selected_X)
            selected_pval = sm.OLS(y,selected_X).fit().pvalues[1:] ## 절편항의 p-value는 뺀다
            max_pval = selected_pval.max()
            if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
                remove_variable = selected_pval.idxmax()
                selected_variables.remove(remove_variable)
            else:
                break
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y,sm.add_constant(x_train_s[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

<ipython-input-99-70811c834742>:13: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-99-70811c834742>:13: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-99-70811c834742>:13: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-99-70811c834742>:13: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 

In [100]:
a = selected_variables
a

['for_4050', 'single_o60', 'single_f_o60', 'popu_o60', '가해_10대이하']

In [101]:
# 모델 학습
y123 = sm.OLS(y1_train, x_train_s[a])
fit_y123 = y123.fit()
fit_y123.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                     q1   R-squared (uncentered):                   0.821
Model:                            OLS   Adj. R-squared (uncentered):              0.816
Method:                 Least Squares   F-statistic:                              146.1
Date:                Thu, 02 Dec 2021   Prob (F-statistic):                    1.44e-57
Time:                        17:48:31   Log-Likelihood:                         -803.32
No. Observations:                 164   AIC:                                      1617.
Df Residuals:                     159   BIC:                                      1632.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
for_4050        -0.0002      0.001     -0.214      0.831      -0.002       0.001
single_o60       0.0092      0.007      1.389      0.167      -0.004       0.022
single_f_o60    -0.0117      0.010     -1.223      0.223      -0.031       0.007
popu_o60         0.0010      0.000      2.916      0.004       0.000       0.002
가해_10대이하        -0.4224      0.402     -1.052      0.295      -1.216       0.371
==============================================================================
Omnibus:                       10.045   Durbin-Watson:                   1.149
Prob(Omnibus):                  0.007   Jarque-Bera (JB):                6.264
Skew:                          -0.322   Prob(JB):                       0.0436
Kurtosis:                       2.291   Cond. No.                     1.04e+04
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.04e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [102]:
# 예측 및 결과
y_123_pred = fit_y123.predict(x_test_s[a])
mean_absolute_error(y1_test, y_123_pred)

28.978083814994395